In [1]:
import json
import re
import os

import pandas as pd
from tqdm import tqdm
tqdm.pandas()

import jieba.analyse

In [2]:
### load data 一次就好

catelog = list() # 目录

print(os.environ["LOCALAPPDATA"])
input_filename = os.environ["LOCALAPPDATA"] + r"\Google\Chrome\User Data\Default\Bookmarks"

with open(input_filename, 'r', encoding='utf-8') as f:
    contents = json.load(f)

C:\Users\adam8\AppData\Local


# contents 結構
```python
{  
    'checksum': <str>,    # 一個檢查用的資訊  
    'roots': {  
        'bookmark_bar': {  
            'children': [<nested with dicts with structure same as this dict>],  
            'date_added': <str with numbers>,  
            'date_modified': <str with numbers>,  
            'guid': <str>,    # example: '00000000-0000-4000-a000-000000000002'  
            'id': <str with numbers>,  
            'name': <str>,  
            'type': <str>    # 有 folder, url  
        },  
        'other': {<same as bookmark_bar>},  
        'synced' {<same as bookmark_bar>}:  
    },  
    sync_metadata: <str>,    # 一大串不知道幹嘛的亂碼  
    version: <int>  
}
```

In [3]:
def get_nodes_info(root, parent_name):
    children = root.pop('children', None)
    root['parent'] = parent_name
    info_list = [root]
    
    if children:
        for child in children:
            info_list.extend(get_nodes_info(child, root['name']))
    
    return info_list

In [4]:
bookmark_bar_info = get_nodes_info(contents['roots']['bookmark_bar'], 'bookmark_bar')
other_info = get_nodes_info(contents['roots']['other'], 'other')
synced_info = get_nodes_info(contents['roots']['synced'], 'synced')

info = bookmark_bar_info + other_info + synced_info

df = pd.DataFrame(info)
print(df.shape)
df.head()

(488, 9)


,date_added,date_modified,guid,id,meta_info,name,parent,type,url
0,13202479594197706,13241369625133688,00000000-0000-4000-a000-000000000002,1,NaN,書籤列,bookmark_bar,folder,NaN
1,13233003241034750,13233003241037911,f0b52077-6793-498a-b8eb-462dfc400922,1884,NaN,2019-nCoV,書籤列,folder,NaN
2,13226238181043741,NaN,7f8a21b9-b559-44ab-b711-c41800d188d1,1885,NaN,Coronavirus_Taiwan,2019-nCoV,url,https://viator.maps.arcgis.com/apps/opsdashboa...
3,13226407607803828,NaN,24a7d7be-2a87-4481-b157-3773db2f4eab,1886,NaN,2019-nCoV | 武漢肺炎地圖,2019-nCoV,url,https://kiang.github.io/2019-nCoV/
4,13227360115873492,NaN,a09ae3f0-3a5f-417c-ad10-423d35e155f3,1888,NaN,程序員硬核勸告：現在還不是出門的時候 - 幫趣,2019-nCoV,url,https://bangqu.com/a7wXH5.html


In [ ]:
jieba.analyse.set_stop_words('meaningless_words.txt')

df['keyword'] = df['name'].progress_apply(jieba.analyse.tfidf)
df

# TODO
1. [x] 加入'tag' by 斷詞
2. [x] 加入母資料夾
3. [ ] 類似的東東們, i.e. recommendation system not yet

In [6]:
### 現在要做出 recommendation system

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\KUROGI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.437 seconds.
Prefix dict has been built successfully.


In [ ]:
### key: 列出不重複的tags，value: 每個tag分別有哪些書籤

keywords = [keyword for keywords in df['keyword'] for keyword in keywords]
keywords = list(set(keywords))
record = {i:[] for i in keywords}

[i for i in df.itertuples()]
# for idx, sublist in enumerate(output['tag']):
#     for item in sublist:
#         record[item].append(output['id'][idx])

# record

In [10]:
# output.iloc[0,:].to_dict() #弄成json傳給網頁
l = [len(l) for l in record.values()]
l = max(l)

# for idx,keys in enumerate(record):
#     print(keys)
# enumerate(record)

In [11]:
### main do things and output to md

bookmark_bar = html_for_node(contents['roots']['bookmark_bar'])
other = html_for_node(contents['roots']['other'])
catelog_str = ''.join(a for a in catelog)

output_file_name = "output_markdown.md"
with open(output_file_name, 'w', encoding='utf-8') as f:
#     f.write(output_file_template.format(catelog=catelog_str, bookmark_bar=bookmark_bar, other=other)) #origin
    f.write(output_file_template.format(other=other))

# other

In [11]:
# def download_thumb_img(url):
#     load_html
#     load_all_img
#     find_max_img

# route = download_thumb_img(googlebookmarkinfo['url'][idx])    # ex: 'C:/example.png'

# 爬一個網址中的所有圖片